In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import requests
import os
import asyncio
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
load_dotenv(override=True)

True

In [4]:
google_api_key = os.getenv('GOOGLE_API_KEY')
gmail_app_password = os.getenv('GMAIL_APP_PASSWORD')
from_email = os.getenv('GMAIL_USER')
to_email = os.getenv('GMAIL_TO')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

Google API Key exists and begins AI


In [5]:
external_client: AsyncOpenAI = AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

llm_model: OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=external_client
)

In [6]:
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects via Gmail SMTP """
    
    subject = "Sales email"
    
    # Create the email
    msg = MIMEText(body, 'html')
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    print(msg)
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        return {"status": "success"}
    except Exception as e:
        return {"status": "failure", "message": str(e)}


# send_email("Hello From Gmail Python")

In [7]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [8]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=llm_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=llm_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=llm_model
)

In [9]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamlining SOC2 Compliance & Audit Readiness with AI

Dear [Prospect Name],

Navigating SOC2 compliance is a significant undertaking, often characterized by manual processes, extensive documentation, and the persistent challenge of maintaining audit readiness. We understand the considerable resources – time, budget, and personnel – that are typically invested, alongside the pressure to secure and uphold certification.

At ComplAI, we've developed an AI-powered SaaS solution specifically designed to transform this process. We help organizations like yours move beyond reactive, manual compliance efforts to a proactive, intelligent, and significantly more efficient system.

Our platform empowers you to:

*   **Automate Evidence Collection:** Leverage AI to seamlessly gather and organize necessary documentation, dramatically reducing manual effort.
*   **Ensure Continuous Compliance:** Monitor your security posture in real-time, identifying gaps and providing actionable insights

In [10]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

CancelledError: 

In [11]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=llm_model
)

In [25]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Confession: We're making SOC2 boring... for your auditor.

Hey [Prospect Name],

I know what you're thinking: "Another cold email." And you're right! But before you hit delete, let me guess: the phrase "SOC2 audit" makes you instinctively reach for a strong drink, or at least wonder if you can pull off faking a sudden, severe case of the Tuesdays.

You're not alone. Most companies treat SOC2 compliance like an annual root canal – necessary, painful, and you usually leave feeling a bit numb. But what if it didn't have to be? What if you could spend less time herding digital cats and more time actually… innovating?

That's where **ComplAI** waltzes in, cape optional, AI-powered for maximum efficiency. We've built the ultimate digital sidekick that eats compliance headaches for breakfast, turning mountains of evidence into molehills of effortless, audit-ready documentation. Our AI doesn't just manage your compliance; it *prepares* you, making your next audit les

In [12]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=llm_model,
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=llm_model,
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=llm_model,
)

sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7f8408630b60>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [13]:


@function_tool
def send_email(body: str):
    """Send out an email with the given body to all sales prospects via Gmail SMTP"""

    subject = "Sales email"

    # Create a multipart message
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = to_email

    # Attach the body (HTML)
    html_part = MIMEText(body, "html")
    msg.attach(html_part)

    print(msg.as_string())  # Debug: see raw message

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        return {"status": "success"}
    except Exception as e:
        return {"status": "failure", "message": str(e)}


In [14]:
print(type(send_email))

<class 'agents.tool.FunctionTool'>


In [28]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")

In [18]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f83de36dbc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f83de36e980>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [16]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model=llm_model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

Content-Type: multipart/alternative; boundary="===============0595577540391659366=="
MIME-Version: 1.0
Subject: Sales email
From: imohsinsheikhani@gmail.com
To: imohsinsheikhani@gmail.com

--===============0595577540391659366==
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

U3ViamVjdDogUHJvYWN0aXZlIFNPQzIgQ29tcGxpYW5jZSAmIEF1ZGl0IFJlYWRpbmVzcyBmb3Ig
W0NvbXBhbnkgTmFtZV0KCkRlYXIgQ0VPLAoKSSB1bmRlcnN0YW5kIHRoYXQgbGVhZGluZyBhIGdy
b3dpbmcgY29tcGFueSBpbnZvbHZlcyBuYXZpZ2F0aW5nIGNyaXRpY2FsIG9wZXJhdGlvbmFsIGNo
YWxsZW5nZXMsIG5vdCBsZWFzdCBhbW9uZyB0aGVtIGVuc3VyaW5nIHJvYnVzdCBkYXRhIHNlY3Vy
aXR5IGFuZCBhY2hpZXZpbmcgZXNzZW50aWFsIGNvbXBsaWFuY2UgY2VydGlmaWNhdGlvbnMgbGlr
ZSBTT0MyLgoKVGhlIHByb2Nlc3Mgb2YgYWNoaWV2aW5nIGFuZCBtYWludGFpbmluZyBTT0MyIGNv
bXBsaWFuY2UgaXMgbm90b3Jpb3VzbHkgY29tcGxleCwgdGltZS1jb25zdW1pbmcsIGFuZCByZXNv
dXJjZS1pbnRlbnNpdmUuIEl0IG9mdGVuIGRpdmVydHMgdmFsdWFibGUgaW50ZXJuYWwgcmVzb3Vy
Y2VzIGFuZCBjYW4gYmVjb21lIGEgYm90dGxlbmVjayBpbiBzZWN1cmluZyBuZX

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [13]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=llm_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=llm_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [22]:


@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given body to all sales prospects via Gmail SMTP"""

    # Create a multipart message
    msg = MIMEText(html_body, 'html')
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = to_email

    print(msg.as_string())  # Debug: see raw message

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        return {"status": "success"}
    except Exception as e:
        return {"status": "failure", "message": str(e)}


In [23]:
tools = [subject_tool, html_tool, send_html_email]

In [24]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=llm_model,
    handoff_description="Convert an email to HTML and send it")

In [25]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f83de36dbc0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f83de36e980>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [26]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=llm_model
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit
Subject: SOC2: Less Root Canal, More Digital Garden?
From: imohsinsheikhani@gmail.com
To: imohsinsheikhani@gmail.com


<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Transforming SOC2 Compliance with ComplAI</title>
    <!-- Preheader text for email clients -->
    <div style="display: none; font-size: 1px; color: #fefefe; line-height: 1px; max-height: 0px; max-width: 0px; opacity: 0; overflow: hidden; mso-hide: all;">
        Tired of SOC2 headaches? Discover how ComplAI makes compliance surprisingly enjoyable.
    </div>
    <style type="text/css">
        /* Client-specific resets */
        body, #bodyTable, #bodyCell {
            height: 100% !important;
            margin: 0;
            padding: 0;
            width: 100% !important;
            font-family: 'Helvetica Neue', Helvetica, Arial,